训练代码参考:
https://github.com/ZHKKKe/MODNet/issues/200
https://www.kaggle.com/code/daggerx/modnet/notebook
预训练模型:
https://drive.google.com/drive/folders/1umYmlCulvIFNaqPjwod1SayFmSRHziyR

In [1]:
import os
import torch
import shutil
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import pandas as pd
from scipy.ndimage.morphology import grey_dilation, grey_erosion
import torch.nn.functional as F
import torchvision.transforms as transforms
from scipy.ndimage import morphology
import cv2
from torch.utils.data import DataLoader
from src.models.modnet import MODNet
from src.trainer import supervised_training_iter
from tqdm import tqdm
import time
import fiftyone as fo

In [2]:
os.chdir("../..")
print(os.getcwd())
# dataset_path = "datasets/objects"
dataset_path = "datasets/open-images-v7/train"
ckpt_path = "pretrained/modnet_photographic_portrait_matting.ckpt"

D:\Projects\MODNet


In [47]:
fo.config.dataset_zoo_dir = r"D:\Projects\MODNet\datasets"
trainset = fo.zoo.load_zoo_dataset(
              "open-images-v7",
              split="train",
              label_types=["segmentations"],
              classes=["Bottle","Dress","Skirt","Swimwear","Miniskirt"],
              max_samples=1000,
          )

Found 100 images, downloading the remaining 900
 100% |███████████████████| 900/900 [4.0m elapsed, 0s remaining, 1.7 files/s]      
Dataset info written to 'D:\Projects\MODNet\datasets\open-images-v7\info.json'
Loading 'open-images-v7' split 'train'
 100% |███████████████| 1000/1000 [2.1m elapsed, 0s remaining, 12.8 samples/s]      
Dataset 'open-images-v7-train-1000' created


In [80]:
from fiftyone import ViewField as vf
trainset_view = trainset.match(
    vf("ground_truth.detections.label").is_subset(
        ["Dress"]
    )
)
trainset_view

Dataset:     open-images-v7-train-1000
Media type:  image
Num samples: 198
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Match(filter={'$expr': {'$setIsSubset': [...]}})

In [51]:
trainset_view.first()

<SampleView: {
    'id': '642be765450f8cb9d9047e5d',
    'media_type': 'image',
    'filepath': 'D:\\Projects\\MODNet\\datasets\\open-images-v7\\train\\data\\00002b368e91b947.jpg',
    'tags': ['train'],
    'metadata': None,
    'ground_truth': <Detections: {

        'detections': [

            <Detection: {


                'id': '642be765450f8cb9d9047e58',


                'attributes': {},


                'tags': [],


                'label': 'Miniskirt',


                'bounding_box': [0.095, 0.683121, 0.261, 0.315287],


                'mask': array([[False, False, False, ..., False, False, False],


                       [False, False, False, ..., False, False, False],


                       [False, False, False, ..., False, False, False],


                       ...,


                       [False, False, False, ...,  True,  True,  True],


                       [False, False, False, ...,  True,  True,  True],


                       [False, False, False, ...,

In [3]:
def cv2_imshow(image):

    # image = cv2.resize(image, (720, 480))

    cv2.imshow("img", image)
    #  add below code
    cv2.waitKey(0)
    cv2.destroyAllWindows()

import matplotlib
import matplotlib.pyplot as plt
def cv2plt(img):
    plt.figure(figsize=(7,7))        # To change the size of figure
    plt.axis('off')
    if np.size(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img,cmap='gray',vmin=0,vmax=255)
    plt.show()

print(cv2plt)

<function cv2plt at 0x0000018F7FA29158>


In [116]:
session = fo.launch_app(trainset_view)

In [127]:
import fiftyone as fo
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms


class OpenImagesDataset(Dataset):
    def __init__(self, dataset, resize_dim, transform=None):
        self.samples = dataset
        self.transform = transform
        self.resize_dim = resize_dim
        self.img_paths = self.samples.values("filepath")

    def __len__(self):
        return len(self.samples)

    def get_trimap(self, alpha):
        # alpha \in [0, 1] should be taken into account
        # be careful when dealing with regions of alpha=0 and alpha=1
        fg = np.array(np.equal(alpha, 255).astype(np.float32))
        unknown = np.array(np.not_equal(alpha, 0).astype(np.float32)) # unknown = alpha > 0
        unknown = unknown - fg
        # image dilation implemented by Euclidean distance transform
        unknown = morphology.distance_transform_edt(unknown==0) <= np.random.randint(1, 20)
        trimap = fg
        trimap[unknown] = 0.5
        return torch.unsqueeze(torch.from_numpy(trimap), dim=0)#.astype(np.uint8)

    def _resize(self, img, trimap=False):
        im = img[None, :, :, :]
        ref_size = self.resize_dim

        # resize image for input
        im_b, im_c, im_h, im_w = im.shape
        if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
            if im_w >= im_h:
                im_rh = ref_size
                im_rw = int(im_w / im_h * ref_size)
            elif im_w < im_h:
                im_rw = ref_size
                im_rh = int(im_h / im_w * ref_size)
        else:
            im_rh = im_h
            im_rw = im_w

        im_rw = im_rw - im_rw % 32
        im_rh = im_rh - im_rh % 32
        if trimap == True:
            im = F.interpolate(im, size=(im_rh, im_rw), mode='nearest')
        else:
            im = F.interpolate(im, size=(im_rh, im_rw), mode='area')
        return im

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        masks = []
        detections = sample["ground_truth"].detections
        for det in detections:
            masks.append(det.mask)
        mask = np.bitwise_or.reduce(masks)

        # Load image
        img = np.asarray(Image.open(img_path).convert("RGB"))
        img = Image.fromarray(img)
        mask = Image.fromarray(mask)
        # Apply transform if specified
        if self.transform:
            img = self.transform(img)
            trimap = self.get_trimap(mask)
            mask = self.transform(mask)

        # Convert mask to torch tensor
        # masks = torch.as_tensor(masks)
        # img = torch.tensor(img, dtype=torch.float32)
        #
        # mask = torch.tensor(mask, dtype=torch.float32)
        # trimap = torch.squeeze(trimap, 1)

        img = self._resize(img)
        mask = self._resize(mask)
        trimap = self._resize(trimap, trimap=True)

        img = torch.squeeze(img, 0)
        mask = torch.squeeze(mask, 0)
        trimap = torch.squeeze(trimap, 1)

        return img, trimap, mask


In [128]:
# torch_trainset = FiftyOneTorchDataset(trainset)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
])
torch_trainset = OpenImagesDataset(trainset_view, 512, transform)

In [129]:
print(trainset_view.first())

<SampleView: {
    'id': '642be767450f8cb9d9047e90',
    'media_type': 'image',
    'filepath': 'D:\\Projects\\MODNet\\datasets\\open-images-v7\\train\\data\\000161230ca61936.jpg',
    'tags': ['train'],
    'metadata': None,
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '642be767450f8cb9d9047e8d',
                'attributes': {},
                'tags': [],
                'label': 'Dress',
                'bounding_box': [0.181875, 0.477486, 0.224375, 0.5206379999999999],
                'mask': array([[False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       ...,
                       [False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False, False],
                       [False, False, False, ..., False, False,

In [5]:
class ModNetDataLoader(Dataset):
    def __init__(self, annotations_file, resize_dim, transform=None):
        self.img_labels =annotations_file
        self.transform=transform
        self.resize_dim=resize_dim

    def __len__(self):
        #return the total number of images
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx,0]
        mask_path = self.img_labels.iloc[idx,1]

        img = np.asarray(Image.open(img_path))

        in_image = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        # cv2_imshow(in_image)

        mask = in_image[:,:,3]

        if len(img.shape)==2:
            img = img[:,:,None]
        if img.shape[2]==1:
            img = np.repeat(img, 3, axis=2)
        elif img.shape[2]==4:
            img = img[:,:,0:3]

        if len(mask.shape)==3:
            mask = mask[:,:, 0]

        #convert Image to pytorch tensor
        img = Image.fromarray(img)
        mask = Image.fromarray(mask)
        if self.transform:
            img = self.transform(img)
            trimap = self.get_trimap(mask)
            mask = self.transform(mask)

        img = self._resize(img)
        mask = self._resize(mask)
        trimap = self._resize(trimap, trimap=True)

        img = torch.squeeze(img, 0)
        mask = torch.squeeze(mask, 0)
        trimap = torch.squeeze(trimap, 1)

        return img, trimap, mask

    def get_trimap(self, alpha):
        # alpha \in [0, 1] should be taken into account
        # be careful when dealing with regions of alpha=0 and alpha=1
        fg = np.array(np.equal(alpha, 255).astype(np.float32))
        unknown = np.array(np.not_equal(alpha, 0).astype(np.float32)) # unknown = alpha > 0
        unknown = unknown - fg
        # image dilation implemented by Euclidean distance transform
        unknown = morphology.distance_transform_edt(unknown==0) <= np.random.randint(1, 20)
        trimap = fg
        trimap[unknown] = 0.5
        return torch.unsqueeze(torch.from_numpy(trimap), dim=0)#.astype(np.uint8)

    def _resize(self, img, trimap=False):
        im = img[None, :, :, :]
        ref_size = self.resize_dim

        # resize image for input
        im_b, im_c, im_h, im_w = im.shape
        if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
            if im_w >= im_h:
                im_rh = ref_size
                im_rw = int(im_w / im_h * ref_size)
            elif im_w < im_h:
                im_rw = ref_size
                im_rh = int(im_h / im_w * ref_size)
        else:
            im_rh = im_h
            im_rw = im_w

        im_rw = im_rw - im_rw % 32
        im_rh = im_rh - im_rh % 32
        if trimap == True:
            im = F.interpolate(im, size=(im_rh, im_rw), mode='nearest')
        else:
            im = F.interpolate(im, size=(im_rh, im_rw), mode='area')
        return im

In [6]:
transformer = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5), (0.5)
            )
        ]
    )
data = ModNetDataLoader(data_csv, 512, transform=transformer)

In [7]:
img, trimap, mask = data[4]
# 打印图像、matte 和 trimap 的形状
print("Image shape:", img.shape)
print("Trimap shape:", trimap.shape)
print("Matte shape:", mask.shape)

Image shape: torch.Size([3, 672, 512])
Trimap shape: torch.Size([1, 672, 512])
Matte shape: torch.Size([1, 672, 512])


In [130]:
img, trimap, mask = torch_trainset[4]
# 打印图像、matte 和 trimap 的形状
print("Image shape:", img.shape)
print("Trimap shape:", trimap.shape)
print("Matte shape:", mask.shape)

Image shape: torch.Size([3, 512, 512])
Trimap shape: torch.Size([1, 576, 256])
Matte shape: torch.Size([1, 512, 512])


In [ ]:
# 展示图像
def img_transpose(img):
    return img.numpy().transpose(2,1,0).transpose(1,0,2)

img_t = img_transpose(img)
trimap_t = img_transpose(trimap)
mask_t = img_transpose(mask)

cv2plt(img_t)
cv2plt(trimap_t)
cv2plt(mask_t)

In [131]:
# 训练加载器
# train_dataloader = DataLoader(data, batch_size=2, shuffle=True)
train_dataloader = DataLoader(torch_trainset, batch_size=2, shuffle=True)
bs = 2  # batch size
lr = 0.01  # learn rate
epochs = 1  # total epochs
step_size = 10  # 学习率将在每 10 个 epoch 之后衰减，epochs < 4 用这行
# step_size=int(0.25 * epochs) # epochs >= 4 用这行

# modnet = torch.nn.DataParallel(MODNet()).cuda()
modnet = torch.nn.DataParallel(MODNet(backbone_pretrained=True)).cuda()
optimizer = torch.optim.SGD(modnet.parameters(), lr=lr, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

for epoch in range(0, epochs):
    print(f"Epoch {epoch + 1} Training: ")
    start_time = time.time()
    total_loss = 0
    batch_count = 0

    train_dataloader_progress = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    for idx, (image, trimap, gt_matte) in train_dataloader_progress:
        semantic_loss, detail_loss, matte_loss = \
            supervised_training_iter(modnet, optimizer, image.cuda(), trimap.cuda(), gt_matte.cuda())
        total_loss += semantic_loss + detail_loss + matte_loss
        batch_count += 1

        # 更新进度条信息
        train_dataloader_progress.set_description(
            f"Batch {idx + 1}: Semantic Loss {semantic_loss:.4f}, Detail Loss {detail_loss:.4f}, Matte Loss {matte_loss:.4f}")

    end_time = time.time()
    avg_loss = total_loss / batch_count
    print(f"Epoch {epoch + 1}/{epochs}, Avg Loss: {avg_loss:.4f}, Time: {end_time - start_time:.2f}s")
    lr_scheduler.step()

torch.save(modnet.state_dict(), "pretrained/my_train/modnet.ckpt")

Epoch 1 Training: 


  0%|          | 0/99 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [1, 928, 512] at entry 0 and [1, 544, 512] at entry 1